In [65]:
import requests,json
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [66]:
URL = 'https://www.monsterindia.com/search/freshers-jobs?searchId=b4215b81-3488-432e-9936-6f2c6c42640c'
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
page = requests.get(URL,headers=agent)

In [67]:
soup = BeautifulSoup(page.content,'html.parser')

In [68]:
job_elems = soup.find_all('div',class_='card-apply-content')

In [69]:
client = MongoClient("mongodb://localhost:27017/")
db = client["job_posting"]
collections = db["job_data"]

In [70]:
for job_elem in job_elems:
    job_title = job_elem.find('h3',class_='medium')

    company_name = job_elem.find('span',class_='company-name')

    job_location = job_elem.find('span',class_='loc')
    a = job_location.text.strip().split(',')
    job_location = []
    for x in a:
        job_location.append(x.strip())

    exp = job_elem.find('div',class_='exp col-xxs-12 col-sm-3 text-ellipsis')

    package = job_elem.find('div',class_='package col-xxs-12 col-sm-4 text-ellipsis')

    job_description = job_elem.find('p',class_='job-descrip')

    skills = job_elem.find_all('label',class_='grey-link')
    skills_req = []
    for x in skills:
        a = x.text.strip().split(',')
        skills_req.append(a[0].strip())

    
    job_data = {'job_title':job_title.text.strip(),'company_name':company_name.text.strip(),'job_location':job_location,'exp':exp.text.strip(),
                   'package':package.text.strip(),'job_description':job_description.text.strip(),'skills':skills_req}
    
    y = json.dumps(job_data)
    x = json.loads(y)
    
    if isinstance(x,list):
        collections.insert_many(x)
    else:
        collections.insert_one(x)
